In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

In [3]:
#from ordinaloss.utils.datasets import ImageFolder, create_transform_pipeline
from ordinaloss.utils.pretrained_models import classification_model_densenet, classification_model_resnet, classification_model_vgg
from ordinaloss.utils.utils import create_ordinal_cost_matrix


from ordinaloss.engine.model_engine import OrdinalEngine
from ordinaloss.utils.datasets import data_load

from ordinaloss.utils.loss_utils import SinimLoss, GirlsLoss, CSCELoss


import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

/home/dsi/itaym/anaconda3/envs/maruloss/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


loaded ordinaloss.utils.pretrained_models
loaded ordinaloss.engine.model_engine
loaded ordinaloss.engine.model_engine
loaded ordinaloss.utils.datasets
loaded ordinaloss.utils.loss_utils


'cuda:0'

In [ ]:
model = classification_model_vgg("vgg19", num_classes = 5)
cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=20)
loaders, _, _ = data_load("../datasets/kneeKL224/", 16)

engine = OrdinalEngine(
    model=model, 
    loss_fn=CSCELoss(cost_matrix), 
    device=device, 
    loaders=loaders,
    use_lr_scheduler=False,
    optimizer_fn=SGD, lr=1.0e-3, weight_decay=5.0e-2
    )

engine.train(n_epochs=25)

In [8]:
from ordinaloss.utils.loss_utils import GirlsLoss, SinimLoss


def my_lambda_scheduler(epoch):
    return (0.9 ** (epoch // 5))


# engine = OrdinalEngine(model, 
#                        loss_fn = CSCELoss(create_ordinal_cost_matrix(5)), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)


# engine = OrdinalEngine(model, 
#                        loss_fn = SinimLoss(), #CSCELoss model, matan, you know the dril.
#                        device = device, #change to gpu, let me know if it doesnt work on GPU.
#                        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
#                        optimizer_fn=SGD, lr = 5.0e-4, weight_decay=5.0e-4)



for diagonal_value in [7,9,13,20]:

    model = classification_model_vgg("vgg19", num_classes = 5)
    cost_matrix = create_ordinal_cost_matrix(5, cost_distance=3, diagonal_value=diagonal_value)
    loaders, _, _ = data_load("../datasets/kneeKL224/", 16)

    engine = OrdinalEngine(
        model=model, 
        loss_fn=CSCELoss(cost_matrix), 
        device=device, 
        loaders=loaders,
        use_lr_scheduler=True, scheduler_lambda_fn = my_lambda_scheduler,
        optimizer_fn=SGD, lr=1.0e-3, weight_decay=5.0e-2
        )

    engine.train(n_epochs=25)

100%|██████████| 104/104 [00:06<00:00, 16.66it/s, accuracy=0.661, batch_size=1656, loss=28.9, mae=0.385]
